In [1]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ff_energy.ffe.utils import read_from_pickle
import jax.numpy as jnp
from ff_energy.ffe.plot import plot_energy_MSE, plot_ff_fit
from ff_energy.ffe.structure import atom_key_pairs
from ff_energy.ffe.potential import LJ, akp_indx
from ff_energy.ffe.ff import FF
from ff_energy.ffe.ff_fit import LJ_bound, load_ff, fit_func, fit_repeat
from ff_energy.ffe.data import pairs_data
from ff_energy.ffe.utils import pickle_output, read_from_pickle, str2int




In [2]:
sig_bound = (0.05, 2.5)
ep_bound = (0.001, 0.5)
LJ_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound))
DE_bound = ((sig_bound), (sig_bound), (ep_bound), (ep_bound), 
            (1, 8), (6, 20))

In [3]:
# ffpkl = "ECOL_pbe0dz_FOE.ff.pkl"

In [4]:
# Parameters
ffpkl = "kmdcm_pbe0dz_clusters.ff.pkl"


#  LJ fit


In [5]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))
_c.data

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


,ECOL,KEY,TOTAL,ELEC,VDW,KEY,M_ENERGY,KEY,n_monomers,C_ENERGY,...,intE,P_intE,m_E_tot,p_m_E_tot,C_ENERGY_kcalmol,DUMMY,k,epol_mike,first_order_energy,epol_pairs
test0,-173.022645,test0,24.14023,-104.54998,20.41138,test0,-1527.133263,test0,20,-1527.276175,...,-88.423162,-77.478222,958362.266640,-958281.958385,-958370.381547,0,0,-197.910658,7.640822,-182.982917
test1,-105.096240,test1,25.25058,-63.45565,8.97296,test1,-1527.135387,test1,20,-1527.221635,...,-53.572842,-53.285800,958360.933743,-958282.584949,-958336.157791,0,1,-113.959761,-5.902676,-111.406093
test2,-144.567121,test2,24.75756,-89.34864,14.41942,test2,-1527.140946,test2,20,-1527.260418,...,-74.628103,-70.885989,958357.445616,-958285.865741,-958360.493844,0,2,-160.484782,-6.037152,-153.110225
test3,-105.558619,test3,21.75948,-62.57541,5.74741,test3,-1527.151295,test3,20,-1527.247454,...,-59.373736,-58.662892,958350.951437,-958292.985598,-958352.359334,0,3,-118.662845,-4.570214,-111.548989
test4,-141.957058,test4,21.69034,-90.29468,12.55821,test4,-1527.150321,test4,20,-1527.270835,...,-74.612483,-73.371184,958351.562845,-958292.418135,-958367.030618,0,4,-155.773808,-12.251942,-151.839749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test495,-156.403481,test495,24.92171,-95.04093,17.54292,test495,-1527.129876,test495,20,-1527.255655,...,-76.747423,-72.611238,958364.391623,-958280.758013,-958357.505436,0,495,-175.806320,-1.718463,-166.329665
test496,-126.839243,test496,15.65968,-75.31144,12.74644,test496,-1527.168887,test496,20,-1527.264530,...,-58.484893,-61.118914,958339.912612,-958304.589688,-958363.074582,0,496,-138.410126,2.169631,-133.079412
test497,-160.377711,test497,20.65947,-90.76807,24.76786,test497,-1527.154064,test497,20,-1527.248923,...,-58.307082,-57.960443,958349.213649,-958294.973785,-958353.280867,0,497,-178.929213,31.250204,-170.886865
test498,-154.210967,test498,21.38907,-87.19819,20.35679,test498,-1527.150078,test498,20,-1527.257280,...,-64.986090,-65.470853,958351.715184,-958293.538913,-958358.525003,0,498,-171.903376,15.363398,-164.288222


In [7]:
_c

FF: LJ water_cluster ELEC harmonic intE (jax_coloumb: True)

In [6]:
_c.targets.mean()

Array(19.06506, dtype=float32)

In [24]:
fit_repeat(_c, 
           10, 
           f"{ffpkl}_LJ",
           bounds = LJ_bound,
           loss="jax",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:174: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [25]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
9,16.203783,317,588,0,True,Optimization terminated successfully.,"[0.050000000000000405, 1.7980929624885533, 0.1...","([[0.050000000000000405, 1.7980929624885533, 0..."
6,16.219980,276,547,0,True,Optimization terminated successfully.,"[0.13860327738606754, 1.7789044434063288, 0.01...","([[0.13860327738606754, 1.7789044434063288, 0...."
5,16.223236,374,691,0,True,Optimization terminated successfully.,"[0.17364307590067213, 1.7811698326994354, 0.01...","([[0.17364307590067213, 1.7811698326994354, 0...."
4,16.231428,373,657,0,True,Optimization terminated successfully.,"[0.23289193152844734, 1.7699237705384943, 0.00...","([[0.23289193152844734, 1.7699237705384943, 0...."
2,16.279417,208,409,0,True,Optimization terminated successfully.,"[0.05204575312453126, 1.842261609031267, 0.5, ...","([[0.05204575312453126, 1.842261609031267, 0.5..."
1,418.809387,40,62,0,True,Optimization terminated successfully.,"[0.05, 0.05000000000000007, 0.001, 0.5]","([[0.05, 0.05000000000000007, 0.001, 0.5], [0...."
7,418.809418,58,169,0,True,Optimization terminated successfully.,"[0.05, 0.05, 0.5, 0.10056543659104933]","([[0.05, 0.05, 0.5, 0.10056543659104933], [0.0..."
0,418.809448,53,154,0,True,Optimization terminated successfully.,"[0.05, 0.05, 0.5, 0.5]","([[0.05, 0.05, 0.5, 0.5], [0.05, 0.05000015893..."
3,418.809448,53,149,0,True,Optimization terminated successfully.,"[0.05, 0.05, 0.5, 0.46895584358815334]","([[0.05, 0.05, 0.5, 0.46895584358815334], [0.0..."
8,418.809448,44,141,0,True,Optimization terminated successfully.,"[0.050000000000000044, 0.05, 0.5, 0.5]","([[0.050000000000000044, 0.05, 0.5, 0.5], [0.0..."


#  DE fit


In [26]:
_c = next(read_from_pickle(f"ff/{ffpkl}"))

In [27]:
fit_repeat(_c, 
           10, 
           "test",
           bounds = DE_bound,
           loss="jax_de",
           quiet='true'
          )

/home/boittier/Documents/phd/ff_energy/ff_energy/ffe/ff_fit.py:174: OptimizeWarning: Unknown solver options: pgtol
  res = minimize(


In [28]:
pd.DataFrame(_c.opt_results).sort_values("fun")

,fun,nit,nfev,status,success,message,x,final_simplex
2,16.179247,873,1422,0,True,Optimization terminated successfully.,"[0.050105844457475485, 1.775936442288697, 0.48...","([[0.050105844457475485, 1.775936442288697, 0...."
5,16.296047,486,863,0,True,Optimization terminated successfully.,"[0.24981973209827044, 1.7419408182262863, 0.01...","([[0.24981973209827044, 1.7419408182262863, 0...."
1,16.305397,438,756,0,True,Optimization terminated successfully.,"[0.07039072161698706, 1.7404361012705065, 0.49...","([[0.07039072161698706, 1.7404361012705065, 0...."
0,17.697798,1136,1826,0,True,Optimization terminated successfully.,"[0.05000140366633622, 2.499967313960714, 0.286...","([[0.05000140366633622, 2.499967313960714, 0.2..."
3,17.950644,825,1355,0,True,Optimization terminated successfully.,"[0.12427892201688796, 2.0197124645237747, 0.47...","([[0.12427892201688796, 2.0197124645237747, 0...."
9,19.247555,706,1181,0,True,Optimization terminated successfully.,"[1.6942631654395086, 1.9021118961322858, 0.001...","([[1.6942631654395086, 1.9021118961322858, 0.0..."
4,20.818403,690,1149,0,True,Optimization terminated successfully.,"[1.543751503450616, 1.5449157833681744, 0.0050...","([[1.543751503450616, 1.5449157833681744, 0.00..."
6,418.809387,54,219,0,True,Optimization terminated successfully.,"[0.05, 0.36104553135892364, 0.2237460600532596...","([[0.05, 0.36104553135892364, 0.22374606005325..."
7,418.809387,43,204,0,True,Optimization terminated successfully.,"[0.05207178730850526, 0.10031999720367063, 0.0...","([[0.05207178730850526, 0.10031999720367063, 0..."
8,418.809387,56,220,0,True,Optimization terminated successfully.,"[0.2371452357449687, 0.05, 0.5, 0.100030834190...","([[0.2371452357449687, 0.05, 0.5, 0.1000308341..."
